In [21]:
print('Good day!!!')

Good day!!!


In [55]:
import pandas as pd

# загрузили файл данных
with open("/home/yurist/Рабочий стол/LINUX/COD/PYTHON/MLLearn/checkins.dat", 'r') as fo:
    lines = fo.readlines()
fo.close()

# убрали все пробелы из строки и заменили разделитель ' | ' на ','
linesCSV = [(' '.join(line.split())).replace(' | ', ',')  for line in lines]



#print( linesCSV[9] )

984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22


In [41]:
//?

SyntaxError: invalid syntax (<ipython-input-41-2266a8cc5790>, line 1)